Instalação do lime

In [ ]:
!pip install lime

Importação de biblioteca

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import lime.lime_tabular

carregar e separar dados

In [ ]:
data = fetch_openml("credit-g", as_frame=True)
df = data.frame

X = df.drop("class", axis=1)
y = df["class"]

X = pd.get_dummies(X)

encoder = LabelEncoder()
y = encoder.fit_transform(y)

Treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

LIME

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X.columns,
    class_names=encoder.classes_,
    mode="classification",
    discretize_continuous=True,
    random_state=42
)

Explicação de acordo com um cliente selecionado

In [ ]:
instance_idx = 5
instance = X_test.iloc[instance_idx]

def run_lime_explanation(instance, explainer, model, num_features=8):
    explanation = explainer.explain_instance(
        instance,
        model.predict_proba,
        num_features=num_features
    )
    return explanation.as_list()

print("\nCrédito do cliente:", encoder.inverse_transform([y_test[instance_idx]]))

explanation1 = run_lime_explanation(instance, explainer, rf)
print("\nExplicação do valor das contribuições para o resultado:")
for feat, contrib in explanation1:
    print(f"{feat}: {round(contrib,8)}")